## Getting boxscore using nba_api

In [ ]:
"""
Goal:
Collect the box scores of every NBA regular game matchup in the 2022-23 season.

Eventually, this will serve to help illustrate the ebs and flows of shooting accuaracy throughout the course of an NBA season.

Could we see whether there are such moments where the league, as a whole, exhibit shooting slumps?

"""

# importing necessaries
from nba_api.live.nba.endpoints import boxscore
import pandas as pd

# Define gameID base value for 2022-23 regular season NBA games 
gameID_base = '00222{}'

# Test boxscore variable
test_box = boxscore.BoxScore('0022200001')

## Creating dictionaries

In [ ]:
# create a working dictionary
box_dict = {'Date':[], 'Home/Away':[]}

# create a blank dictionary, adopting the box score statistic's keys 
stats_key_dictionary = dict.fromkeys(test_box.get_dict()['game']['homeTeam']['statistics'].keys(), [])

# update box_dict with keys from ['statistics].keys() call
starting_dict.update(box_dictionary)

## Creating loop to fill dictionary

In [ ]:
# loop through each matchup and collect home and away box score data 
# test loop to go through first five games of 2022-23 season
for i in range(1,5):
    gameID = str(i).zfill(5) #avoids leading zero issues
    print(gameID) #check gameID result
    
    # calls on API to retrieve the boxscores data of the first five games
    box_first_five = boxscore.BoxScore(gameID_base.format(gameID))
    
    # create variable that holds the match's datetime
    date_dict = {"Date": [box_first_five.get_dict()['game']['gameEt']]}
    print(date_dict) #print as necesssary
    print(type(date_dict))
    
    # calling to get home and away box score data
    away_box_dict = box_first_five.get_dict()['game']['awayTeam']['statistics']
    home_box_dict = box_first_five.get_dict()['game']['homeTeam']['statistics']
    
    # have new dictionary read into dataframe
    away_df = pd.DataFrame.from_dict(away_box, orient='index').T
    print(away_df) #print away dataframe as necessary
    home_df = pd.DataFrame.from_dict(home_box, orient='index').T
    print(home_df) #print home dataframe as necessary
    

## Great alternative, but was getting timeout errors

In [ ]:
# Import the nba_api and pandas modules
import nba_api
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2

# Define the season
season = "2022-23"

# Get all the regular season games for the season
games = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season").get_data_frames()[0]

# Create an empty dataframe to store the box score data
box_score_df = pd.DataFrame()

# Loop through the games and get the box score for each game
for game in games.itertuples():
    # Get the game id
    game_id = game.GAME_ID
    # Get the box score
    box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
    # Append the box score data to the dataframe
    box_score_df = pd.concat([box_score_df, box_score], ignore_index=True)
    print(game_id)
    

# Save the dataframe to a file or a database of your choice (you can use any format you want)
box_score_df.to_csv("Spreadsheets/box_score_all.csv", index=False)